Load Variables

In [8]:
%run "scripts/load_daily_variables.py"

Loading Variables: 100%|██████████| 26/26 [01:06<00:00,  2.55s/it]


High Beta with Recent Elevated Volatility and Activity

In [9]:
hbeta_vol_regime_5day = [sym for sym in symbols if 
                    (sym in results_finvizsearch.loc[results_finvizsearch['Beta'] > 2]['Ticker'].to_list()) and 
                    (symbols[sym].ATRs_Traded.replace(np.inf, np.nan).dropna().iloc[-5:] >= 1.5).any() and
                    (symbols[sym].RVol.replace(np.inf, np.nan).dropna().iloc[-5:] >= 1.5).any()
                    ]
hbeta_vol_regime_20day = [sym for sym in symbols if 
                    (sym in results_finvizsearch.loc[results_finvizsearch['Beta'] > 2]['Ticker'].to_list()) and 
                    (symbols[sym].ATRs_Traded.replace(np.inf, np.nan).dropna().iloc[-20:] >= 1.5).any() and
                    (symbols[sym].RVol.replace(np.inf, np.nan).dropna().iloc[-20:] >= 1.5).any() and
                    (sym not in hbeta_vol_regime_5day)
                    ]

In [10]:
mask = daily_quant_rating_df.index.isin(hbeta_vol_regime_5day)
col = daily_quant_rating_df.columns[-2]
(
daily_quant_rating_df.loc[mask, col]
.sort_values(ascending=False)
.head(50)
)

Symbol
LASR    4.980122
GLXY    4.888379
TPC     4.870031
APP     4.784404
FLNC    4.778287
EBS     4.703364
ADPT    4.694190
BFLY    4.614679
DAVE    4.581040
NVTS    4.503058
TXG     4.464578
SFIX    4.350136
CDNA    4.202997
PACB    3.952316
VIK     3.786104
AMRC    3.647139
ONDS    3.494524
IREN    3.483915
LMND    3.477413
AMKR    3.477070
CIFR    3.465092
RKT     3.454141
DPRO    3.437372
RDDT    3.428816
HOOD    3.426762
TMDX    3.399042
ASTS    3.393224
CTMX    3.380561
AMPX    3.359343
NXT     3.291923
SHOP    3.156057
ACHR    3.136893
U       3.136208
UMAC    3.135524
AFRM    3.132786
NVAX    3.072211
RIOT    3.061944
KOPN    3.043121
SPIR    3.039699
UCTT    3.035250
PTON    3.034223
WRBY    3.015058
TREE    2.993498
MSTR    2.988364
AEHR    2.933607
LIND    2.907255
PDYN    2.866530
RYAM    2.814511
JOBY    2.813826
TBCH    2.808693
Name: 2026-01-16, dtype: float64

In [11]:
mask = daily_quant_rating_df.index.isin(hbeta_vol_regime_20day)
col = daily_quant_rating_df.columns[-2]
(
daily_quant_rating_df.loc[mask, col]
.sort_values(ascending=False)
.head(50)
)

Symbol
GRAL    4.963303
CRDO    4.943425
HUT     4.896024
CTRN    4.847095
BTSG    4.773700
GAP     4.602446
AS      4.590214
CCL     4.561162
OS      4.515291
PSIX    4.448229
CUK     4.393733
DAN     3.974114
LSPD    3.816076
INGM    3.791553
NCLH    3.660763
FDMT    3.540872
SKYT    3.497604
PRAX    3.497262
BE      3.492813
REAL    3.492129
APLD    3.491444
SLS     3.491102
COMP    3.490196
RKLB    3.488364
SYRE    3.448323
SOFI    3.438741
LC      3.425736
JMIA    3.414100
SMTC    3.406571
CENX    3.378850
NBIS    3.350103
ADEA    3.337440
Z       3.284314
IMNM    3.247433
SONO    3.204997
BKSY    3.173511
OMER    3.168378
CVNA    3.157769
INOD    3.131075
AMSC    3.102669
NAGE    3.099932
HTZ     3.090007
SAIA    3.064682
SPCE    3.053046
AUR     3.046201
EXPI    3.029412
RUN     3.021903
OPEN    3.019608
QXO     3.006160
PTGX    2.991102
Name: 2026-01-16, dtype: float64

YTD Performance vs. Quant Rating

In [14]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook_connected"  # loads plotly.js via CDN
quant_col = daily_quant_rating_df.columns[-2]
quant_rating = daily_quant_rating_df[quant_col].reset_index().rename(columns={quant_col:'Quant Rating'})
perf = results_finvizsearch[['Ticker', 'Performance (YearToDate)']]
df = quant_rating.merge(perf, left_on='Symbol', right_on='Ticker', how='inner')
# df = df.loc[df.Ticker.isin(fu.sector_industry_member_search('Basic Materials', level='sector'))]
# Assign color based on logic:
# quant rating > 4.8 & Performance < 50 -> green
# quant rating < 3 & Performance > 50 -> red
# else -> gray
def assign_color(row):
    if (row['Quant Rating'] > 4.8) and (row['Performance (YearToDate)'] < 50):
        return 'green'
    elif (row['Quant Rating'] < 3) and (row['Performance (YearToDate)'] > 50):
        return 'red'
    elif (row['Quant Rating'] > 2.5) and (row['Quant Rating'] < 4) and (row['Performance (YearToDate)'] > 50):
        return 'yellow'
    else:
        return 'gray'

df['Color'] = df.apply(assign_color, axis=1)

px.scatter(
    df.assign(
        Color_Label=df['Color'].map({'gray': 'uncertain', 'green': 'undervalued', 'red': 'overvalued', 'yellow': 'worth watching'})
    ),
    x='Performance (YearToDate)',
    y='Quant Rating',
    hover_data=['Ticker'],
    color='Color_Label',
    color_discrete_map={'uncertain': 'gray', 'undervalued':'green', 'overvalued':'red', 'worth_watching': 'yellow'},
    category_orders={'Color_Label': ['uncertain', 'undervalued', 'overvalued', 'worth_watching']},
    labels={'Color_Label': 'Category'},
    title='YTD Performance vs. Quant Rating'
)